<a href="https://colab.research.google.com/github/Deepanshu2601/Deepanshumachinelearning/blob/main/Ass4ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
url='https://books.toscrape.com/'
r=requests.get(url)
soup=BeautifulSoup(r.content,'html.parser')
data=[]
nums = {"One":1, "Two":2, "Three":3, "Four":4, "Five":5}
for d in soup.find_all('li',attrs={'class':"col-xs-6 col-sm-4 col-md-3 col-lg-3"}):
    img = d.find('img')
    title = img['alt']
    stars = d.find('p',attrs={'class':'star-rating'})
    stars = nums[stars['class'][1]]
    price = d.find('p',attrs={'class':'price_color'}).text
    availability = d.find('p',attrs={'class':'instock availability'}).text
    obj = {"Title":title, "Rating":stars, "Price":price}
    data.append(obj)

df = pd.DataFrame(data)
df

,Title,Rating,Price
0,A Light in the Attic,3,£51.77
1,Tipping the Velvet,1,£53.74
2,Soumission,1,£50.10
3,Sharp Objects,4,£47.82
4,Sapiens: A Brief History of Humankind,5,£54.23
5,The Requiem Red,1,£22.65
6,The Dirty Little Secrets of Getting Your Dream...,4,£33.34
7,The Coming Woman: A Novel Based on the Life of...,3,£17.93
8,The Boys in the Boat: Nine Americans and Their...,4,£22.60
9,The Black Maria,1,£52.15


In [6]:
!pip install selenium

In [7]:
def get_default_chrome_options():
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    return options

from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Chrome()
driver.get("https://quotes.toscrape.com/js/")
options = get_default_chrome_options()
options.add_experimental_option("detach", True)
quotes = driver.find_elements(By.CLASS_NAME, "text")
data = []
for q in quotes:
    print(q.text)
    data.append(q.text)
driver.quit()

SessionNotCreatedException: Message: session not created: Chrome instance exited. Examine ChromeDriver verbose log to determine the cause.; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
#0 0x557c37269aca <unknown>
#1 0x557c36cb5cdb <unknown>
#2 0x557c36cf07b9 <unknown>
#3 0x557c36cec215 <unknown>
#4 0x557c36d3ceb6 <unknown>
#5 0x557c36d3c5d6 <unknown>
#6 0x557c36cfaaca <unknown>
#7 0x557c36cfb7d1 <unknown>
#8 0x557c37230a99 <unknown>
#9 0x557c37233a6c <unknown>
#10 0x557c37219d29 <unknown>
#11 0x557c37234665 <unknown>
#12 0x557c372016a3 <unknown>
#13 0x557c372567b8 <unknown>
#14 0x557c37256993 <unknown>
#15 0x557c37268a63 <unknown>
#16 0x7f23e3efbac3 <unknown>


In [ ]:
!pip install selenium webdriver-manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException, TimeoutException, StaleElementReferenceException
)
from webdriver_manager.chrome import ChromeDriverManager
import csv

URL = "https://quotes.toscrape.com/js/"

def create_driver(headless=False):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")  # newer headless mode
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--disable-dev-shm-usage")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=opts)
    return driver

def scrape_all_pages():
    driver = create_driver(headless=True)
    wait = WebDriverWait(driver, 10)
    driver.get(URL)

    results = []
    page_index = 1

    while True:
        try:
            # Wait for quotes to be present on the current page
            wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "quote")))
        except TimeoutException:
            print(f"Timed out waiting for quotes on page {page_index}. Stopping.")
            break

        # Collect all quotes on the page
        quotes = driver.find_elements(By.CLASS_NAME, "quote")
        for q in quotes:
            try:
                text = q.find_element(By.CLASS_NAME, "text").text.strip()
                author = q.find_element(By.CLASS_NAME, "author").text.strip()
                tags = [t.text.strip() for t in q.find_elements(By.CLASS_NAME, "tag")]
                results.append({"quote": text, "author": author, "tags": ";".join(tags)})
            except (StaleElementReferenceException, NoSuchElementException):
                # element disappeared between find and access — skip it
                continue

        # Try to find and click the Next button. If not found, we're done.
        try:
            next_link = driver.find_element(By.CSS_SELECTOR, "li.next a")
        except NoSuchElementException:
            print("No next button found — reached last page.")
            break

        # Click and wait for URL to change (safe navigation)
        prev_url = driver.current_url
        try:
            next_link.click()
        except Exception:
            # fallback to JS click if normal click fails
            driver.execute_script("arguments[0].click();", next_link)

        try:
            wait.until(EC.url_changes(prev_url))
        except TimeoutException:
            # URL didn't change within timeout, but maybe content changed — attempt to continue
            print(f"Warning: URL didn't change after clicking Next on page {page_index}.")
        page_index += 1

    driver.quit()
    return results

def save_to_csv(rows, filename="quotes.csv"):
    keys = ["quote", "author", "tags"]
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        writer.writerows(rows)
    print(f"Saved {len(rows)} rows to {filename}")

if __name__ == "__main__":
    data = scrape_all_pages()
    save_to_csv(data)

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

def scrape_imdb_top250():
    """
    Scrapes IMDB Top 250 Movies list and saves to CSV
    """

    # Configure Chrome options
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run in background
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

    # Initialize the driver
    print("Initializing Chrome driver...")
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # Navigate to IMDB Top 250
        url = "https://www.imdb.com/chart/top/"
        print(f"Navigating to {url}...")
        driver.get(url)

        # Wait for the page to load
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ipc-metadata-list")))

        print("Page loaded successfully. Extracting movie data...")

        # Lists to store data
        ranks = []
        titles = []
        years = []
        ratings = []

        # Find all movie items
        movie_items = driver.find_elements(By.CSS_SELECTOR, "li.ipc-metadata-list-summary-item")

        print(f"Found {len(movie_items)} movies. Processing...")

        for idx, item in enumerate(movie_items, 1):
            try:
                # Extract rank
                rank = idx

                # Extract title
                title_element = item.find_element(By.CSS_SELECTOR, "h3.ipc-title__text")
                title_text = title_element.text
                # Remove the rank prefix (e.g., "1. The Shawshank Redemption" -> "The Shawshank Redemption")
                title = title_text.split('. ', 1)[1] if '. ' in title_text else title_text

                # Extract year
                metadata = item.find_elements(By.CSS_SELECTOR, "span.cli-title-metadata-item")
                year = metadata[0].text if metadata else "N/A"

                # Extract rating
                rating_element = item.find_element(By.CSS_SELECTOR, "span.ipc-rating-star--imdb")
                rating_text = rating_element.get_attribute("aria-label")
                # Extract numeric rating from "IMDb rating: 9.3"
                rating = rating_text.split(": ")[1] if ": " in rating_text else "N/A"

                # Append to lists
                ranks.append(rank)
                titles.append(title)
                years.append(year)
                ratings.append(rating)

                # Print progress every 50 movies
                if idx % 50 == 0:
                    print(f"Processed {idx} movies...")

            except Exception as e:
                print(f"Error processing movie {idx}: {str(e)}")
                continue

        print(f"Successfully extracted data for {len(titles)} movies.")

        # Create DataFrame
        df = pd.DataFrame({
            'Rank': ranks,
            'Movie Title': titles,
            'Year of Release': years,
            'IMDB Rating': ratings
        })

        # Export to CSV
        csv_filename = 'imdb_top250.csv'
        df.to_csv(csv_filename, index=False, encoding='utf-8')
        print(f"\nData saved to '{csv_filename}'")

        # Display first 10 rows
        print("\nFirst 10 movies:")
        print(df.head(10).to_string(index=False))

        # Display summary statistics
        print(f"\nTotal movies scraped: {len(df)}")
        print(f"Average rating: {df['IMDB Rating'].astype(float).mean():.2f}")

        return df

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

    finally:
        # Close the browser
        driver.quit()
        print("\nBrowser closed.")

if __name__ == "__main__":
    print("=" * 60)
    print("IMDB Top 250 Movies Scraper")
    print("=" * 60)

    df = scrape_imdb_top250()

    if df is not None:
        print("\n✓ Scraping completed successfully!")
    else:
        print("\n✗ Scraping failed. Please check the error messages above.")

IMDB Top 250 Movies Scraper
Initializing Chrome driver...


SessionNotCreatedException: Message: session not created: Chrome instance exited. Examine ChromeDriver verbose log to determine the cause.; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
#0 0x5cb0952d8aca <unknown>
#1 0x5cb094d24cdb <unknown>
#2 0x5cb094d5f7b9 <unknown>
#3 0x5cb094d5b215 <unknown>
#4 0x5cb094dabeb6 <unknown>
#5 0x5cb094dab5d6 <unknown>
#6 0x5cb094d69aca <unknown>
#7 0x5cb094d6a7d1 <unknown>
#8 0x5cb09529fa99 <unknown>
#9 0x5cb0952a2a6c <unknown>
#10 0x5cb095288d29 <unknown>
#11 0x5cb0952a3665 <unknown>
#12 0x5cb0952706a3 <unknown>
#13 0x5cb0952c57b8 <unknown>
#14 0x5cb0952c5993 <unknown>
#15 0x5cb0952d7a63 <unknown>
#16 0x7c1349f38ac3 <unknown>


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import re

def scrape_weather_data():
    """
    Scrapes weather information for top world cities from timeanddate.com
    """

    # Configure Chrome options
    chrome_options = Options()
    # chrome_options.add_argument('--headless')  # Uncomment for headless mode
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)

    # Initialize the driver
    print("Initializing Chrome driver...")
    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    try:
        # Navigate to the weather page
        url = "https://www.timeanddate.com/weather/"
        print(f"Navigating to {url}...")
        driver.get(url)

        # Wait for page to load completely
        print("Waiting for page to load completely...")
        time.sleep(5)

        # Scroll down to load all content
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
        time.sleep(2)

        print("Page loaded. Extracting weather data...")

        # Lists to store data
        cities = []
        temperatures = []
        conditions = []

        # Find all table rows
        all_rows = driver.find_elements(By.CSS_SELECTOR, "table tr")

        print(f"Found {len(all_rows)} table rows. Processing...")

        for row in all_rows:
            try:
                # Get all cells in this row
                all_cells = row.find_elements(By.TAG_NAME, "td")

                if len(all_cells) < 2:
                    continue

                # Each row contains multiple cities
                # Pattern: city link, time, weather+temp, wind | city link, time, weather+temp, wind | ...
                # We need to process cells in groups

                i = 0
                while i < len(all_cells):
                    try:
                        # Check if this cell contains a city link
                        city_links = all_cells[i].find_elements(By.CSS_SELECTOR, "a[href*='/weather/']")

                        if not city_links:
                            i += 1
                            continue

                        city_name = city_links[0].text.strip()

                        if not city_name or len(city_name) < 2:
                            i += 1
                            continue

                        # Next cell should be time (class="r"), skip it
                        # The cell after that should contain weather image and temperature
                        temperature = "N/A"
                        condition = "N/A"

                        # Look ahead for the weather data cell (usually 2 cells ahead)
                        if i + 2 < len(all_cells):
                            weather_cell = all_cells[i + 2]

                            # Extract weather condition from image alt attribute
                            try:
                                weather_img = weather_cell.find_element(By.TAG_NAME, "img")
                                condition = weather_img.get_attribute("alt").strip()

                                # Remove any extra text after period (e.g., "Snow flurries. Overcast. Cold." -> "Snow flurries")
                                if '.' in condition:
                                    condition = condition.split('.')[0].strip()
                            except:
                                pass

                        # Temperature is in the next cell (class="rbi")
                        if i + 3 < len(all_cells):
                            temp_cell = all_cells[i + 3]
                            cell_text = temp_cell.text.strip()

                            # Extract temperature (format like "-4 °C" or "25 °C")
                            # Remove &nbsp; and extract number with degree symbol
                            cell_text = cell_text.replace('\xa0', ' ')  # Replace &nbsp;
                            temp_match = re.search(r'(-?\d+)\s*°[CF]?', cell_text)
                            if temp_match:
                                temperature = temp_match.group(1) + "°C"

                        # Only add if we got at least temperature or condition
                        if temperature != "N/A" or condition != "N/A":
                            cities.append(city_name)
                            temperatures.append(temperature)
                            conditions.append(condition)

                            if len(cities) % 25 == 0:
                                print(f"Processed {len(cities)} cities...")

                        # Move to next potential city (usually 4 cells ahead: city, time, weather, wind)
                        i += 4

                    except Exception as e:
                        i += 1
                        continue

            except Exception as e:
                continue

        print(f"\nSuccessfully extracted data for {len(cities)} cities.")

        if not cities:
            print("\n⚠ No weather data could be extracted.")
            print("The page structure may have changed or dynamic content didn't load.")
            return None

        # Create DataFrame
        df = pd.DataFrame({
            'City Name': cities,
            'Temperature': temperatures,
            'Weather Condition': conditions
        })

        # Remove duplicates (keep first occurrence)
        df = df.drop_duplicates(subset=['City Name'], keep='first')

        # Export to CSV
        csv_filename = 'weather.csv'
        df.to_csv(csv_filename, index=False, encoding='utf-8')
        print(f"\nData saved to '{csv_filename}'")

        # Display sample results
        print("\nSample Weather Data (first 20 cities):")
        print(df.head(20).to_string(index=False))

        print(f"\n{'='*60}")
        print(f"Total cities scraped: {len(df)}")
        print(f"Cities with temperature data: {len(df[df['Temperature'] != 'N/A'])}")
        print(f"Cities with weather condition data: {len(df[df['Weather Condition'] != 'N/A'])}")

        # Show some example weather conditions
        unique_conditions = df[df['Weather Condition'] != 'N/A']['Weather Condition'].unique()
        print(f"\nUnique weather conditions found: {len(unique_conditions)}")
        print(f"Examples: {', '.join(list(unique_conditions)[:10])}")

        return df

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

    finally:
        # Close the browser
        print("\nClosing browser...")
        driver.quit()
        print("Browser closed.")

if __name__ == "__main__":
    print("=" * 60)
    print("Weather Information Scraper - TimeAndDate.com")
    print("=" * 60)
    print()

    df = scrape_weather_data()

    if df is not None and len(df) > 0:
        print("\n✓ Scraping completed successfully!")
    else:
        print("\n✗ Scraping failed. Please check the error messages above.")
        print("\nTroubleshooting tips:")
        print("1. Make sure you have a stable internet connection")
        print("2. The website may have anti-scraping measures")
        print("3. Try running without headless mode to see what's happening")
        print("4. Check if the website structure has changed")

Weather Information Scraper - TimeAndDate.com

Initializing Chrome driver...


SessionNotCreatedException: Message: session not created: Chrome instance exited. Examine ChromeDriver verbose log to determine the cause.; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
#0 0x564c4c2e8aca <unknown>
#1 0x564c4bd34cdb <unknown>
#2 0x564c4bd6f7b9 <unknown>
#3 0x564c4bd6b215 <unknown>
#4 0x564c4bdbbeb6 <unknown>
#5 0x564c4bdbb5d6 <unknown>
#6 0x564c4bd79aca <unknown>
#7 0x564c4bd7a7d1 <unknown>
#8 0x564c4c2afa99 <unknown>
#9 0x564c4c2b2a6c <unknown>
#10 0x564c4c298d29 <unknown>
#11 0x564c4c2b3665 <unknown>
#12 0x564c4c2806a3 <unknown>
#13 0x564c4c2d57b8 <unknown>
#14 0x564c4c2d5993 <unknown>
#15 0x564c4c2e7a63 <unknown>
#16 0x7d24c3ba8ac3 <unknown>
